Los objetivos son: encontrar posibles relaciones entre las variables que conforman el dataset, extraer algunas conclusiones que ayuden a analizar los factores que afectan al precio de los alquileres, y estudiar distintos estadísticos que nos permitan tener un mayor conocimiento de cómo se comporta el sector del alquiler turístico privado en el estado de Nueva York.

El dataset recoge los datos de Airbnb relativos al tramo 2008 - 2019. Fuente: Kaggle

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import folium

La primera parte del análisis va a consistir en tener una visión global respecto de cómo están distribuidos los datos con los que vamos a trabajar y, en caso de que proceda, corregir y limpiar aquellas variables que, por contener valores erróneos o nulos, así lo necesiten.

Importamos el archivo y los convertimos a un dataframe de Pandas y observamos el número de datos que contiene cada una de las columnas

In [2]:
df_NY = pd.read_csv('../data/raw/AB_NYC_2019.csv')


df_NY.count()

id                                48895
name                              48879
host_id                           48895
host_name                         48874
neighbourhood_group               48895
neighbourhood                     48895
latitude                          48895
longitude                         48895
room_type                         48895
price                             48895
minimum_nights                    48895
number_of_reviews                 48895
last_review                       38843
reviews_per_month                 38843
calculated_host_listings_count    48895
availability_365                  48895
dtype: int64

Obtenemos un pequeño resumen de los principales estadísticos del dataset que nos permita identificar posibles outliers o valores que, por estar demasiado alejados del global, pueden interferir de forma negativa en el modelo y que éste no refleje una imagen fiel de la realidad

In [3]:
df_NY.describe()

,id,host_id,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
count,4.889500e+04,4.889500e+04,48895.000000,48895.000000,48895.000000,48895.000000,48895.000000,38843.000000,48895.000000,48895.000000
mean,1.901714e+07,6.762001e+07,40.728949,-73.952170,152.720687,7.029962,23.274466,1.373221,7.143982,112.781327
std,1.098311e+07,7.861097e+07,0.054530,0.046157,240.154170,20.510550,44.550582,1.680442,32.952519,131.622289
min,2.539000e+03,2.438000e+03,40.499790,-74.244420,0.000000,1.000000,0.000000,0.010000,1.000000,0.000000
25%,9.471945e+06,7.822033e+06,40.690100,-73.983070,69.000000,1.000000,1.000000,0.190000,1.000000,0.000000
50%,1.967728e+07,3.079382e+07,40.723070,-73.955680,106.000000,3.000000,5.000000,0.720000,1.000000,45.000000
75%,2.915218e+07,1.074344e+08,40.763115,-73.936275,175.000000,5.000000,24.000000,2.020000,2.000000,227.000000
max,3.648724e+07,2.743213e+08,40.913060,-73.712990,10000.000000,1250.000000,629.000000,58.500000,327.000000,365.000000


En el caso de la columna "minimum_nights", no parece muy lógico que el valor sea de 1250 noches. Esta idea se refuerza aún más con la información que aporta el cuartil 3, en el cual podemos ver que en un 75% de los casos el valor para esta variable es 5 o inferior.

Otro de los valores que pueden afectar de forma negativa es el que se refiere al precio máximo. Si tenemos en cuenta que la media se sitúa en 152 dólares, y que el 75% de los precios pisos tienen, como máximo, un precio de 175 dólares, lo más adecuado es prescindir del piso que tiene como precio máximo 10.000 dólares.

El número de registros que tienen un precio superior a 1000 dólares es de 239. Es decir, un 0,48% del total del Dataset. Se tratan, por tanto, de muy pocos registros con unos valores que difieren mucho del resto, y que debemos evitar si queremos conseguir un modelo que se adecúe a la realidad lo máximo posible

In [4]:
df_NY[df_NY["price"]>2000].count()

id                                86
name                              86
host_id                           86
host_name                         86
neighbourhood_group               86
neighbourhood                     86
latitude                          86
longitude                         86
room_type                         86
price                             86
minimum_nights                    86
number_of_reviews                 86
last_review                       35
reviews_per_month                 35
calculated_host_listings_count    86
availability_365                  86
dtype: int64

Lo mismo ocurre con el número de registros que tienen como valor una cifra superior a 75 noches como límite mínimo de estancia

In [5]:
df_NY[df_NY["minimum_nights"]>100].count()

id                                174
name                              173
host_id                           174
host_name                         174
neighbourhood_group               174
neighbourhood                     174
latitude                          174
longitude                         174
room_type                         174
price                             174
minimum_nights                    174
number_of_reviews                 174
last_review                        89
reviews_per_month                  89
calculated_host_listings_count    174
availability_365                  174
dtype: int64

Vamos a eliminar estos valores extremos que los definimos como outliers, valores fuera del rango

In [7]:
# eliminamos los valores cuyo valor de precio sea mayor que 2000
df_2_NY=df_NY[df_NY['price']>2000]
df_2_NY.describe()

,id,host_id,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
count,8.600000e+01,8.600000e+01,86.000000,86.000000,86.000000,86.000000,86.000000,35.000000,86.000000,86.000000
mean,2.014155e+07,7.668502e+07,40.735700,-73.975903,4250.976744,23.953488,3.569767,0.731143,2.813953,196.197674
std,1.145228e+07,8.438469e+07,0.046727,0.039290,2216.885706,50.582933,9.158141,1.228401,3.592514,142.215977
min,3.636730e+05,2.132660e+05,40.583630,-74.097300,2010.000000,1.000000,0.000000,0.020000,1.000000,0.000000
25%,1.238491e+07,4.849923e+06,40.714363,-73.998270,2592.500000,1.000000,0.000000,0.145000,1.000000,83.000000
50%,2.227991e+07,3.910141e+07,40.733365,-73.982745,3361.000000,2.000000,0.000000,0.320000,1.000000,180.000000
75%,3.040641e+07,1.531804e+08,40.767230,-73.964165,5000.000000,30.000000,2.000000,0.925000,2.000000,361.750000
max,3.618920e+07,2.712487e+08,40.886710,-73.770690,10000.000000,365.000000,69.000000,6.150000,12.000000,365.000000
